In [1]:
import multiprocessing as mp

if __name__ == "__main__":
	
    proc = mp.current_process() ### 현재 vscode의 pid를 보여주는것이다. 
    print(proc.name)
    print(proc.pid)

MainProcess
88904


In [2]:
import multiprocessing as mp
import os
import time

def worker():
    time.sleep(2)  ## 이거해주면 terminal에서 보기 쉽다. 
    print(os.getpid())
    print("Subprocess End")

if __name__== "__main__":
	
    proc = mp.Process(name="Subprocess", target=worker)  ### 이렇게 하면 새로운 프로세스가 생겨난다. 자식프로세스가 생겨나는 것. 
    print(proc.pid) ## 여기다 하는건 의미없다. 
    proc.start()
    proc.join()

None
89222
Subprocess End


---
> spawn: process생성 속도는 fork보다 살짝 느리다. child process는 최소한의 자원만 승계받는다. Unix와 Windows에서 모두 사용가능하다. Windows에서 multiprocess사용시 기본옵션이다.
>
> fork: child process는 parent의 모든 자원을 매우 효과적으로 승계받는다. Unix에서 multiprocess사용시 기본 옵션이다.



In [3]:
import multiprocessing as mp
import os
import time

def worker():
    time.sleep(2)  ## 이거해주면 terminal에서 보기 쉽다. 
    print(os.getpid())
    print("Subprocess End")

if __name__== "__main__":
	### 이렇게 기본적으로 ƒork fork로 해야 새로운 프로세스가 생겨난다. 터미널에서도 보이고. 
    ctx = mp.get_context('fork')
    proc = ctx.Process(name='fork_test', target=worker)
    proc.start()
    proc.join()


89442
Subprocess End


In [4]:
import multiprocessing as mp
import os
import time

def worker():
    time.sleep(2)  ## 이거해주면 terminal에서 보기 쉽다. 
    print(os.getpid())
    print("Subprocess End")

if __name__== "__main__":
	### 이렇게 spawn으로 바꿔버리면 안생김....... 
    ctx = mp.get_context('spawn')
    proc = ctx.Process(name='spawn_test', target=worker)
    proc.start()
    proc.join()

In [6]:
from multiprocessing import Pool
import numpy as np

def f(x):
    print(os.getpid())  ### 이렇게 함수안에다가 즉, worker안에다가 넣어놔야지 해당 process의 pid를 알 수 있다. 
    return x*x

if __name__ == '__main__':
	
    x = np.arange(25)
    p = Pool(5)
    print(p.map(f, [x[i*5:i*5+5] for i in range(5)]))
    p.close() ## POOL객체를 join으로 꺼주려면 p.close()를 먼저해주어야 한다. 이게 귀찮아서 아래에서는 pool 안쓰고
    ## for문으로 process를 list에 담아서 하나씩 끄는듯하다. 
    p.join() ## join안해주면 좀비프로세스가 된다. 

89721
89722
89723
89724
89725
[array([ 0,  1,  4,  9, 16]), array([25, 36, 49, 64, 81]), array([100, 121, 144, 169, 196]), array([225, 256, 289, 324, 361]), array([400, 441, 484, 529, 576])]


In [7]:
import os
import time
from multiprocessing import Process, freeze_support


def task(idx, count):
    print(f"PID : {os.getpid()}")
    logic = sum([i ** 2 for i in range(count)])
    return idx, logic


if __name__ == "__main__":

    job = [("첫 번째", 10 ** 7), ("두 번째", 10 ** 7), ("세 번째", 10 ** 7), ("네 번째", 10 ** 7)]

    start = time.time()
    
    process = []
    for idx, count in job:
        p = Process(target=task, args=(idx, count))
        p.start()
        process.append(p)

    for p in process:
        p.join() ### 프로세스가 끝날때까지 기다렸다가 종료해주는것. 

    print(f"End Time : {time.time() - start}s")

    start = time.time()

    for idx, count in job:
        task(idx, count)  ### 이렇게 process 생성안하면 jupyter process와 동일한 이름으로 처리된다. 

    print(f"End Time : {time.time() - start}s")

PID : 90115
PID : 90118
PID : 90119
PID : 90120
End Time : 2.5227129459381104s
PID : 88904
PID : 88904
PID : 88904
PID : 88904
End Time : 9.184499263763428s


In [8]:
from multiprocessing import Pool
import numpy as np

def f(x):
    return x*x

if __name__ == '__main__':
    num = 20000
    x = np.arange(num)
    p = Pool(5)
    print(p.map(f, [x[i*5:i*5+4000] for i in range(int(num/5))]))
    p.close()
    p.join()

[array([       0,        1,        4, ..., 15976009, 15984004, 15992001]), array([      25,       36,       49, ..., 16016004, 16024009, 16032016]), array([     100,      121,      144, ..., 16056049, 16064064, 16072081]), array([     225,      256,      289, ..., 16096144, 16104169, 16112196]), array([     400,      441,      484, ..., 16136289, 16144324, 16152361]), array([     625,      676,      729, ..., 16176484, 16184529, 16192576]), array([     900,      961,     1024, ..., 16216729, 16224784, 16232841]), array([    1225,     1296,     1369, ..., 16257024, 16265089, 16273156]), array([    1600,     1681,     1764, ..., 16297369, 16305444, 16313521]), array([    2025,     2116,     2209, ..., 16337764, 16345849, 16353936]), array([    2500,     2601,     2704, ..., 16378209, 16386304, 16394401]), array([    3025,     3136,     3249, ..., 16418704, 16426809, 16434916]), array([    3600,     3721,     3844, ..., 16459249, 16467364, 16475481]), array([    4225,     4356,     4489, 

In [9]:
import os
os.cpu_count()

10

In [10]:
def count(process_name):
	for i in range(1, 100001):

         print(process_name, i)

import time
from multiprocessing import Pool
if __name__=="__main__":
    start_time = time.time()
    
    p_list = ["proc_1", "proc_2", "proc_3", "proc_4"]
    pool = Pool(processes = 4)
    pool.map(count, p_list)
    pool.close()
    pool.join()
    
    print(time.time() - start_time)

proc_3 1
proc_2 1
proc_1 1
proc_4 1
proc_2 2
proc_4 2
proc_3 2
proc_1 2
proc_4 3
proc_2 3
proc_1 3
proc_3 3
proc_4 4
proc_2 4
proc_3 4
proc_1 4
proc_2 5
proc_4 5
proc_3 5
proc_1 5
proc_2 6
proc_3 6
proc_4 6
proc_1 6
proc_2 7
proc_4 7
proc_3 7
proc_1 7
proc_3 8
proc_2 8
proc_4 8
proc_1 8
proc_4 9
proc_2 9
proc_3 9
proc_1 9
proc_2 10
proc_3 10
proc_4 10
proc_1 10
proc_2 11
proc_3 11
proc_1 11
proc_4 11
proc_2 12
proc_3 12
proc_1 12
proc_4 12
proc_1 13
proc_3 13
proc_2 13
proc_4 13
proc_1 14
proc_4 14
proc_3 14
proc_2 14
proc_1 15
proc_4 15
proc_3 15
proc_2 15
proc_1 16
proc_4 16
proc_3 16
proc_2 16
proc_1 17
proc_4 17
proc_2 17
proc_3 17
proc_1 18
proc_4 18
proc_3 18
proc_2 18
proc_1 19
proc_4 19
proc_3 19
proc_2 19
proc_1 20
proc_4 20
proc_3 20
proc_2 20
proc_1 21
proc_4 21
proc_3 21
proc_1 22
proc_2 21
proc_4 22
proc_3 22
proc_1 23
proc_2 22
proc_4 23
proc_1 24
proc_2 23
proc_3 23
proc_4 24
proc_1 25
proc_3 24
proc_2 24
proc_4 25
proc_1 26
proc_3 25
proc_2 25
proc_4 26
proc_3 26
proc_1

In [37]:
from multiprocessing import Process
import time

def func(name):
    start = time.time()
    print(os.getpid())
    print(name, " started")
    time.sleep(2)
    delta_t = time.time()-start
    print(name, " ended, took ",delta_t)

if __name__ == "__main__":
    # Without Process
    start = time.time()
    func("No Process 1")
    func("No Process 2")
    delta_t = time.time() - start
    print("Without Process : ",delta_t,"s")
    
    # With Process
    start = time.time()
    proc1 = Process(target=func,args=('process1',))
    proc2 = Process(target=func,args=('process2',))
    proc1.start()
    proc2.start()
    proc1.join()
    proc2.join()
    delta_t = time.time()- start
    print("With Process : ",delta_t,"s")

No Process 1  started
No Process 1  ended, took  2.0052378177642822
No Process 2  started
No Process 2  ended, took  2.0026581287384033
Without Process :  4.008953094482422 s
process1  started
process2  started
process1  ended, took  2.0060880184173584
process2  ended, took  2.004225969314575
With Process :  2.0367658138275146 s


In [35]:
from multiprocessing import Process, Queue

def f(q, l:list):
    q.put(l)

if __name__ == '__main__':
    q = Queue()
    p1 = Process(target=f, args=(q,[42, None, 'process 1']))
    p2 = Process(target=f, args=(q,[31, None, 'process 2']))
    p1.start()
    print(q.get())    # prints "[42, None, 'process 1']"
    p1.join()

    p2.start()
    print(q.get())    # prints "[31, None, 'process 2']"
    p2.join()

[42, None, 'process 1']
[31, None, 'process 2']
